# ta_qt_scraper

Code describing the scraping of the tripadvisor site.

In [1]:
import sys  
from PyQt4.QtGui import *  
from PyQt4.QtCore import *  
from PyQt4.QtWebKit import *  
from lxml import html 
import re
import time
import timeout_decorator

In [2]:
class Render(QWebPage):  
    def __init__(self, url):  
        self.app = QApplication(sys.argv)  
        QWebPage.__init__(self)  
        self.loadFinished.connect(self._loadFinished)  
        self.mainFrame().load(QUrl(url))  
        self.app.exec_()  

    def _loadFinished(self, result):  
        self.frame = self.mainFrame()  
        self.app.quit() 
    
    def update_url(self, url):
        self.mainFrame().load(QUrl(url))
        self.app.exec_()  
        

In [3]:
url="http://www.tripadvisor.com/Hotels-g33851-New_Haven_Connecticut-Hotels.html"

In [4]:
#Create the render object:
r = Render(url)

#result is a QString:
result = r.frame.toHtml()

#QString should be converted to string before processed by lxml
formatted_result = str(result.toAscii())

#Next build lxml tree from formatted_result
tree = html.fromstring(formatted_result)

In [5]:
len(tree)

2

In [6]:
#Now using correct Xpath we are fetching URL of archives
hotels_div = tree.xpath('//*[@id="ACCOM_OVERVIEW"]/div/div/div/div')
titles_div = tree.xpath('//*[@class="listing_title"]')

In [7]:
hotels_div

[<Element div at 0x10adfaba8>,
 <Element div at 0x10adfac00>,
 <Element div at 0x10adfac58>,
 <Element div at 0x10adfacb0>,
 <Element div at 0x10adfad08>,
 <Element div at 0x10adfad60>,
 <Element div at 0x10adfadb8>,
 <Element div at 0x10adfae10>,
 <Element div at 0x10adfae68>,
 <Element div at 0x10adfaec0>,
 <Element div at 0x10adfaf18>,
 <Element div at 0x10adfaf70>,
 <Element div at 0x10adfafc8>,
 <Element div at 0x10ae24050>,
 <Element div at 0x10ae240a8>,
 <Element div at 0x10ae24100>,
 <Element div at 0x10ae24158>]

In [8]:
titles_div

[<Element div at 0x10ae241b0>,
 <Element div at 0x10ae24208>,
 <Element div at 0x10ae24260>,
 <Element div at 0x10ae242b8>,
 <Element div at 0x10ae24310>,
 <Element div at 0x10ae24368>,
 <Element div at 0x10ae243c0>,
 <Element div at 0x10ae24418>,
 <Element div at 0x10ae24470>,
 <Element div at 0x10ae244c8>,
 <Element div at 0x10ae24520>,
 <Element div at 0x10ae24578>,
 <Element div at 0x10ae245d0>]

In [9]:
r.app.quit()

In [9]:
r.app.exit()

In [10]:
r = 0

In [11]:
r

0

In [12]:
url

'http://www.tripadvisor.com/Hotels-g33851-New_Haven_Connecticut-Hotels.html'

In [ ]:
s = Render(url)

In [9]:
hotel_names = []
links = []

for title in titles_div:
    print(title.text_content())
    hotel_names.append(title.text_content())
    print(title.xpath('a/@href')[0])
    links.append(title.xpath('a/@href')[0])
    print('*'*50)

Omni New Haven Hotel at YaleSpecial Offer12% Off or Free Parking! 
/Hotel_Review-g33851-d119851-Reviews-Omni_New_Haven_Hotel_at_Yale-New_Haven_Connecticut.html
**************************************************
The Study at Yale
/Hotel_Review-g33851-d1217689-Reviews-The_Study_at_Yale-New_Haven_Connecticut.html
**************************************************
Omni New Haven Hotel at YaleSpecial Offer12% Off or Free Parking! 
/Hotel_Review-g33851-d119851-Reviews-Omni_New_Haven_Hotel_at_Yale-New_Haven_Connecticut.html
**************************************************
New Haven Hotel
/Hotel_Review-g33851-d85912-Reviews-New_Haven_Hotel-New_Haven_Connecticut.html
**************************************************
Courtyard New Haven at YaleSpecial OfferDouble Rewards Points! 
/Hotel_Review-g33851-d83697-Reviews-Courtyard_New_Haven_at_Yale-New_Haven_Connecticut.html
**************************************************
New Haven Premiere Hotel and Suites
/Hotel_Review-g33851-d85908-Reviews-Ne

In [10]:
links

['/Hotel_Review-g33851-d119851-Reviews-Omni_New_Haven_Hotel_at_Yale-New_Haven_Connecticut.html',
 '/Hotel_Review-g33851-d1217689-Reviews-The_Study_at_Yale-New_Haven_Connecticut.html',
 '/Hotel_Review-g33851-d119851-Reviews-Omni_New_Haven_Hotel_at_Yale-New_Haven_Connecticut.html',
 '/Hotel_Review-g33851-d85912-Reviews-New_Haven_Hotel-New_Haven_Connecticut.html',
 '/Hotel_Review-g33851-d83697-Reviews-Courtyard_New_Haven_at_Yale-New_Haven_Connecticut.html',
 '/Hotel_Review-g33851-d85908-Reviews-New_Haven_Premiere_Hotel_and_Suites-New_Haven_Connecticut.html',
 '/Hotel_Review-g33851-d119885-Reviews-Duncan_Hotel-New_Haven_Connecticut.html',
 '/Hotel_Review-g33851-d85971-Reviews-La_Quinta_Inn_Suites_New_Haven-New_Haven_Connecticut.html',
 '/Hotel_Review-g33851-d225916-Reviews-Days_Inn_New_Haven-New_Haven_Connecticut.html',
 '/Hotel_Review-g33851-d119904-Reviews-Atlantic_Motel-New_Haven_Connecticut.html',
 '/Hotel_Review-g33851-d83702-Reviews-Regal_Inn-New_Haven_Connecticut.html',
 '/Hotel_Rev

## Now to scrape the reviews from a specific hotel

In [11]:
base_url = 'http://www.tripadvisor.com'
url = base_url+links[0]
url

'http://www.tripadvisor.com/Hotel_Review-g33851-d119851-Reviews-Omni_New_Haven_Hotel_at_Yale-New_Haven_Connecticut.html'

In [12]:
#Create the render object:
r.update_url(url)

#result is a QString:
result = r.frame.toHtml()

#QString should be converted to string before processed by lxml
formatted_result = str(result.toAscii())

#Next build lxml tree from formatted_result
tree = html.fromstring(formatted_result)

In [13]:
# example of how to access specific divs within divs
# sel.xpath('//div[@id="REVIEWS"]/div/div[contains(@class, "review")]/div[@class="col2of2"]/div[@class="innerBubble"]')

In [14]:
full_reviews = tree.xpath('//div[contains(@class, "reviewSelector")]')
# users = tree.xpath('//div[@class="col1of2"]')

In [15]:
full_reviews

[<Element div at 0x109ed9158>,
 <Element div at 0x109ed91b0>,
 <Element div at 0x109ed9208>,
 <Element div at 0x109ed9260>,
 <Element div at 0x109ed92b8>,
 <Element div at 0x109ed9310>,
 <Element div at 0x109ed9368>,
 <Element div at 0x109ed93c0>,
 <Element div at 0x109ed9418>,
 <Element div at 0x109ed9470>]

In [15]:
for fullrev in full_reviews:
    # user name:
    member_info = fullrev.xpath('div/div[contains(@class, "col1of2")]/div[contains(@class, "member_info")]')[0]
    usrnm = member_info.xpath('div/div[contains(@class, "username mo")]')
    review = fullrev.xpath('div/div[@class="col2of2"]/div[@class="innerBubble"]')[0]
    title = review.xpath('div/div[contains(@class, "quote")]')[0].text_content().strip()[1:-1]
    rating = review.xpath('div/div[contains(@class, "rating")]/span/img/@alt')[0].split(' ')[0]
    date = review.xpath('div/div[contains(@class, "rating")]/span[contains(@class, "ratingDate")]/@title')[0]
    rev = review.xpath('div/div[contains(@class, "entry")]')[0].text_content().strip().replace("\n", "")
    if len(usrnm) > 0:
        #location = member_info.xpath('div[1]')[0].text_content()
        print('Username: {}'.format(str(usrnm[0].text_content()).strip()))
    else:
        print('Username: A Trip Advisor Member')
    
    location = member_info.xpath('div[contains(@class, "location")]')
    if len(location) > 0:
        print('Location: {}'.format(str(location[0].text_content()).strip()))
    else:
        print('Location: ')
    
    print('full review_id: {}'.format(fullrev.xpath('@id')[0]))
    try:
        rev_id = re.search('review_(\d+)$', fullrev.xpath('@id')[0]).group(1)
    except AttributeError:
        rev_id = ''

    print('review_id: {}'.format(rev_id))
    print('Title: {}'.format(title))
    print('Rating: {}'.format(rating))
    print('Date: {}'.format(date))
    print('Review:')
    print(rev)
    print('*'*50)


Username: Katelyn F
Location: Long Beach Island, New Jersey
full review_id: review_310707970
review_id: 310707970
Title: Perfect location.
Rating: 5
Date: September 15, 2015
Review:
This was the first stop on our honeymoon road trip, and it was absolutely perfect. You can't beat the location next to Yale and right downtown. Our room was super comfortable. The staff was extremely accommodating and really helped us out when we accidentally we left something important in the room after checkout.
**************************************************
Username: SweetlifeChicago
Location: Chicago
full review_id: review_310654721
review_id: 310654721
Title: Conveniently located
Rating: 3
Date: September 15, 2015
Review:
Very convenient location, walking distance to Yale, restaurants, etc. Rooms are OK and clean but felt outdated.Small fridge and coffee maker below TV. Check-in/check-out very fast. Had a view on parking and port. 15th floor was very quiet, didn't notice any noise from the street. 

In [16]:
len(tree.xpath('//a[contains(@class, "next")]'))
tree.xpath('//a[contains(@class, "next")]/@href')[0]

'/Hotel_Review-g33851-d119851-Reviews-or10-Omni_New_Haven_Hotel_at_Yale-New_Haven_Connecticut.html#REVIEWS'

### Now scraping all pages of the reviews:

In [17]:
@timeout_decorator.timeout(5, use_signals=True)
def return_results(formatted_result, base_url, page):
    #Next build lxml tree from formatted_result
    tree = html.fromstring(formatted_result)
    
    print('Tree returned!')
    print(tree.xpath('//div[contains(@id, "emailOnlySignupWrap")]'))

    full_reviews = tree.xpath('//div[contains(@class, "reviewSelector")]')

    page_usernames = []
    page_locations = []
    page_titles = []
    page_ratings = []
    page_dates = []
    page_reviews = []
    page_review_ids = []

    for fullrev in full_reviews:
        # user name:
        member_info = fullrev.xpath('div/div[contains(@class, "col1of2")]/div[contains(@class, "member_info")]')[0]
        usrnm = member_info.xpath('div/div[contains(@class, "username mo")]')
        review = fullrev.xpath('div/div[@class="col2of2"]/div[@class="innerBubble"]')[0]
        title = review.xpath('div/div[contains(@class, "quote")]')[0].text_content().strip()[1:-1]
        rating = review.xpath('div/div[contains(@class, "rating")]/span/img/@alt')[0].split(' ')[0]
        date = review.xpath('div/div[contains(@class, "rating")]/span[contains(@class, "ratingDate")]/@title')
        if len(date) > 0:
            date = date[0]
        else:
            date = review.xpath('div/div[contains(@class, "rating")]/span[contains(@class, "ratingDate")]')[0].text_content()
        rev = review.xpath('div/div[contains(@class, "entry")]')[0].text_content().strip().replace("\n", "")
        if len(usrnm) > 0:
            #location = member_info.xpath('div[1]')[0].text_content()
            print('Username: {}'.format(str(usrnm[0].text_content()).strip()))
            page_usernames.append(str(usrnm[0].text_content()).strip())
        else:
            print('Username: A Trip Advisor Member')
            page_usernames.append('')

        location = member_info.xpath('div[contains(@class, "location")]')
        if len(location) > 0:
            # print('Location: {}'.format(str(location[0].text_content()).strip()))
            page_locations.append(location[0].text_content().strip())
        else:
            # print('Location: ')
            page_locations.append('')

        print('Title: {}'.format(title.encode('utf-8')))
#         print('Rating: {}'.format(rating))
#         print('Date: {}'.format(date))
#         print('Review:')
#         print(rev)
        page_titles.append(title)
        page_ratings.append(ratings)
        page_dates.append(date)
        page_reviews.append(rev)
        try:
            rev_id = re.search('review_(\d+)$', fullrev.xpath('@id')[0]).group(1)
        except AttributeError:
            rev_id = ''
        page_review_ids.append(rev_id)

#         print('*'*50)

    if len(tree.xpath('//a[contains(@class, "next")]')) > 0:
        url = base_url+tree.xpath('//a[contains(@class, "next")]/@href')[0]
        more_reviews = True
        page += 1
        print('url and page updated.')
    else:
        more_reviews = False
    
    ret_dict = {'usrnms': page_usernames,
                'locs': page_locations,
                'ttls': page_titles,
                'rtngs': page_ratings,
                'dts': page_dates,
                'rvws': page_reviews,
                'revids': page_review_ids,
               'url': url,
               'more_reviews': more_reviews,
               'page': page}
    return ret_dict


In [28]:
usernames = []
locations = []
titles = []
ratings = []
dates = []
reviews = []
review_ids = []

base_url = 'http://www.tripadvisor.com'
url = base_url+links[0]
url


'http://www.tripadvisor.com/Hotel_Review-g33851-d119851-Reviews-Omni_New_Haven_Hotel_at_Yale-New_Haven_Connecticut.html'

In [25]:
page = 1
r.update_url(url)

#result is a QString:
result = r.frame.toHtml()

#QString should be converted to string before processed by lxml
formatted_result = str(result.toAscii())

ret_dict = return_results(formatted_result, base_url, page)


Tree returned!
[]
Username: Katelyn F
Title: Perfect location.
Username: SweetlifeChicago
Title: Conveniently located
Username: Oswaldo A
Title: Best option in new haven
Username: SLR879
Title: Good for a short stay
Username: Valerie P
Title: Perfect stay
Username: tibird77
Title: Pet friendly, clean and convenient
Username: michaelsU5133PY
Title: Nice hotel in a great location
Username: Tami T
Title: New haven 20k
Username: cdecdd28
Title: MPC meet and train
Username: Sandra D
Title: Beautiful View of Yale
url and page updated.


In [26]:
ret_dict['url']

'http://www.tripadvisor.com/Hotel_Review-g33851-d119851-Reviews-or10-Omni_New_Haven_Hotel_at_Yale-New_Haven_Connecticut.html#REVIEWS'

In [ ]:
usernames = []
locations = []
titles = []
ratings = []
dates = []
reviews = []
review_ids = []

base_url = 'http://www.tripadvisor.com'
url = base_url+links[0]
url
more_reviews = True
page = 1
while more_reviews:
    #print('*'*50)
    print('*'*50)
    print('Now on page {}'.format(page))
    #print('*'*50)
    print('*'*50)
    #Create the render object:
    r.update_url(url)

    #result is a QString:
    result = r.frame.toHtml()

    #QString should be converted to string before processed by lxml
    formatted_result = str(result.toAscii())

    try:
        ret_dict = return_results(formatted_result, base_url, page)
        usernames.append(ret_dict['usrnms'])
        locations.append(ret_dict['locs'])
        titles.append(ret_dict['ttls'])
        ratings.append(ret_dict['rtngs'])
        dates.append(ret_dict['dts'])
        reviews.append(ret_dict['rvws'])
        review_ids.append(ret_dict['revids'])
        url = ret_dict['url']
        more_reviews = ret_dict['more_reviews']
        page = ret_dict['page']
    except:
        print('Timed out! Trying that page again...' )


**************************************************
Now on page 1
**************************************************
Tree returned!
[]
Username: Katelyn F
Title: Perfect location.
Username: SweetlifeChicago
Title: Conveniently located
Username: Oswaldo A
Title: Best option in new haven
Username: SLR879
Title: Good for a short stay
Username: Valerie P
Title: Perfect stay
Username: tibird77
Title: Pet friendly, clean and convenient
Username: michaelsU5133PY
Title: Nice hotel in a great location
Username: Tami T
Title: New haven 20k
Username: cdecdd28
Title: MPC meet and train
Username: Sandra D
Title: Beautiful View of Yale
url and page updated.
**************************************************
Now on page 2
**************************************************
Tree returned!
[]
Username: Keith T
Title: Great hotel and location
Username: David S
Title: Boarders and more.
Username: Rose H
Title: One night stay!
Username: Sean713
Title: Best Hotel in the City
Username: Dunmy N
Title: Great st

In [2]:
import time
import timeout_decorator

@timeout_decorator.timeout(5, use_signals=True)
def mytest():
    print "Start"
    for i in range(1,10):
        time.sleep(1)
        print "%d seconds have passed" % i


In [3]:
try:
    mytest()
except:
    print('It worked!')

Start
1 seconds have passed
2 seconds have passed
3 seconds have passed
4 seconds have passed
It worked!


In [6]:
help(r)

Help on Render in module __main__ object:

class Render(PyQt4.QtWebKit.QWebPage)
 |  Method resolution order:
 |      Render
 |      PyQt4.QtWebKit.QWebPage
 |      PyQt4.QtCore.QObject
 |      sip.wrapper
 |      sip.simplewrapper
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, url)
 |  
 |  update_url(self, url)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from PyQt4.QtWebKit.QWebPage:
 |  
 |  acceptNavigationRequest(...)
 |      QWebPage.acceptNavigationRequest(QWebFrame, QNetworkRequest, QWebPage.NavigationType) -> bool
 |  
 |  action(...)
 |      QWebPage.action(QWebPage.WebAction) -> QAction
 |  
 |  applicationCacheQuotaExceeded = <unbound signal applicationCacheQuotaExceeded>
 |  bytesReceived(...)
 |      QWebPage.bytesReceived() -> int
 |  
 |  chooseFile(...)
 |      QWebPage.chooseFile(QWebFrame, QString) -> QString
 |  
 |  contentsChanged = <unbound signal contentsChanged>
 |  cre